# Imports

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import networkit as nk
import tqdm
import numpy as np
import glob
import time
import json

from sklearn.metrics import ndcg_score
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import kendalltau, spearmanr

# Load data

In [6]:
wikipedia_clickstream_df = pd.read_csv(glob.glob("../data/wikipedia_clickstream/*.tsv")[0], sep="\t", on_bad_lines="skip", header=None, names=["prev", "curr", "type", "n"])
wikipedia_clickstream_df = wikipedia_clickstream_df.loc[wikipedia_clickstream_df["type"]!="external", :]
wikipedia_clickstream_df = wikipedia_clickstream_df[:10**5]

print(wikipedia_clickstream_df.head())

                                                 prev  \
5                                   Defense_of_Harbin   
6                      Japanese_invasion_of_Manchuria   
10  List_of_Super_Nintendo_Entertainment_System_games   
11                                             Mode_7   
12                                  List_of_THQ_games   

                       curr  type   n  
5                 Ding_Chao  link  22  
6                 Ding_Chao  link  75  
10  Mohawk_&_Headphone_Jack  link  36  
11  Mohawk_&_Headphone_Jack  link  13  
12  Mohawk_&_Headphone_Jack  link  15  


## Node ID and label mapping

In [7]:
id_label_mapping = {id: label for id, label in enumerate(list(set(wikipedia_clickstream_df["prev"].to_list() + wikipedia_clickstream_df["curr"].to_list())))}
label_id_mapping = {label: id for id, label in id_label_mapping.items()}

# Generate networkit graph

In [8]:
g = nk.Graph(directed=False)

for row in wikipedia_clickstream_df[["prev", "curr"]].to_records(index=False).tolist():
    g.addEdge(label_id_mapping[row[0]], label_id_mapping[row[1]], addMissing=True)

g.removeSelfLoops()

print("Number of nodes: ", g.numberOfNodes())
print("Number of edges: ", g.numberOfEdges())

Number of nodes:  99253
Number of edges:  100000


In [9]:
len(id_label_mapping.keys())

99253

# Network metrics

In [15]:
results = {
            "dataset": "wikipedia_clickstream",
            "directed": False,
            "nodes": g.numberOfNodes(),
            "edges": g.numberOfEdges()
        }

# Density
results["density"] = nk.graphtools.density(g)
print(f"Density: ", results["density"])

# Average clustering coefficient
results["average_clustering_coefficient"] = nk.globals.ClusteringCoefficient().avgLocal(g, 10**6) 
print(f"Average clustering coefficient: ", results["average_clustering_coefficient"])

# Diameter
diameter = nk.distance.Diameter(g, algo=nk.distance.DiameterAlgo.Exact, nSamples=10**5)
diameter.run()
results["diameter"] = diameter.getDiameter() 
print(f"Diameter: ", results["diameter"])

Density:  0.0015375862970309208
Average clustering coefficient:  0.0
Diameter:  (4, 0)


# Centrality measures

In [16]:
def get_degree_centrality(g):
    start_time = time.process_time()
    
    degree = nk.centrality.DegreeCentrality(g)
    degree.run()
    
    end_time = time.process_time()
    
    return degree, (end_time - start_time)

def get_closeness_centrality(g):
    start_time = time.process_time()
    
    closeness = nk.centrality.Closeness(g, True, nk.centrality.ClosenessVariant.Generalized)
    closeness.run()
    
    end_time = time.process_time()
    
    return closeness, (end_time - start_time)

def get_topk_closeness_centrality(g, first_heu=False, second_heu=False, k=5):
    start_time = time.process_time()

    topk_closeness = nk.centrality.TopCloseness(g, k=k, first_heu=first_heu, sec_heu=second_heu)
    topk_closeness.run()
    
    end_time = time.process_time()

    return topk_closeness, (end_time - start_time)

centrality = {}
results["time_elapsed"] = {}

# Degree centrality
centrality["degree"], results["time_elapsed"]["degree"] = get_degree_centrality(g)

# Closeness centrality
centrality["closeness"], results["time_elapsed"]["closeness"] = get_closeness_centrality(g)

# Topk closeness centrality
ks = [5, 10, 50, 100, int(results["nodes"]/2)]

centrality["topkcloseness_0"] = {}
centrality["topkcloseness_1"] = {}
results["time_elapsed"]["topkcloseness_0"] = {}
results["time_elapsed"]["topkcloseness_1"] = {}

for k in ks:
    centrality["topkcloseness_0"][k], results["time_elapsed"]["topkcloseness_0"][k] = get_topk_closeness_centrality(g, False, False, k)
    centrality["topkcloseness_1"][k], results["time_elapsed"]["topkcloseness_1"][k] = get_topk_closeness_centrality(g, False, True, k)

In [17]:
print(results)

{'dataset': 'wikipedia_clickstream', 'directed': False, 'nodes': 1141, 'edges': 1000, 'density': 0.0015375862970309208, 'average_clustering_coefficient': 0.0, 'diameter': (4, 0), 'time_elapsed': {'degree': 0.10293856999999207, 'closeness': 0.19883283999999435, 'topkcloseness_0': {5: 0.44589460800000325, 10: 0.41140032500001666, 50: 0.47931287100001896, 100: 0.47791061399999535, 570: 0.505843021000004}, 'topkcloseness_1': {5: 0.4422009649999836, 10: 0.5268179120000127, 50: 0.5562596980000194, 100: 0.5313926990000084, 570: 0.5703592319999871}}}


# Experiments

## Preprocess

In [18]:
normalised_scores = {}
nodes = {}

# Degree centrality
scaler = MinMaxScaler()
normalised_scores["degree"] = scaler.fit_transform(np.array([row[1] for row in centrality["degree"].ranking()]).reshape(-1, 1)).flatten()
nodes["degree"] = [row[0] for row in centrality["degree"].ranking()]

# Closeness centrality
scaler = MinMaxScaler()
normalised_scores["closeness"] = scaler.fit_transform(np.array([row[1] for row in centrality["closeness"].ranking()]).reshape(-1, 1)).flatten()
nodes["closeness"] = [row[0] for row in centrality["closeness"].ranking()]

# Topk closeness centrality
normalised_scores["topkcloseness_0"], normalised_scores["topkcloseness_1"] = {}, {}
nodes["topkcloseness_0"], nodes["topkcloseness_1"] = {}, {}

for k in ks:
    scaler = MinMaxScaler()
    normalised_scores["topkcloseness_0"][k] = scaler.fit_transform(np.array(centrality["topkcloseness_0"][k].topkScoresList()).reshape(-1, 1)).flatten()
    nodes["topkcloseness_0"][k] = centrality["topkcloseness_0"][k].topkNodesList()

    scaler = MinMaxScaler()
    normalised_scores["topkcloseness_1"][k] = scaler.fit_transform(np.array(centrality["topkcloseness_1"][k].topkScoresList()).reshape(-1, 1)).flatten()
    nodes["topkcloseness_1"][k] = centrality["topkcloseness_1"][k].topkNodesList()

## LabelBinarizer function

In [19]:
def get_label_binarized(labels, node_count, k):
    b = np.full((k, node_count), 0, dtype="byte")
    b[np.arange(k), labels] = 1
    
    return b

## NDCG degree to (top-k) closeness

In [20]:
ndcg_scores = {}

for k in ks[:-1]:
    ndcg_scores[k] = {}
    
    for centrality_measure in ["closeness", "topkcloseness_0", "topkcloseness_1"]:
        
        y_true = get_label_binarized(nodes["degree"][:k], results["nodes"], k)
        sample_weight = normalised_scores["degree"][:k]
        
        if "topk" in centrality_measure:
            y_score = get_label_binarized(nodes[centrality_measure][k], results["nodes"], k)
            
        else:
            y_score = get_label_binarized(nodes[centrality_measure][:k], results["nodes"], k)

        ndcg_scores[k][centrality_measure] = ndcg_score(y_true, y_score, sample_weight=sample_weight)
    
results["ndcg_degree"] = ndcg_scores

print(pd.DataFrame(ndcg_scores).T)

     closeness  topkcloseness_0  topkcloseness_1
5     0.452127         0.452127         0.452127
10    0.349976         0.349976         0.349976
50    0.251025         0.251025         0.251025
100   0.238444         0.238444         0.238444


## NDCG closeness to (top-k) closeness

In [21]:
ndcg_scores = {}

for k in ks[:-1]:
    ndcg_scores[k] = {}
    
    y_score = get_label_binarized(nodes["closeness"][:k], results["nodes"], k)
    sample_weight = normalised_scores["closeness"][:k]
    
    for centrality_measure in ["topkcloseness_0", "topkcloseness_1"]:
        y_true = get_label_binarized(nodes[centrality_measure][k], results["nodes"], k)
            
        ndcg_scores[k][centrality_measure] = ndcg_score(y_true, y_score, sample_weight=sample_weight)
    
results["ndcg_closeness"] = ndcg_scores

print(pd.DataFrame(ndcg_scores).T)

     topkcloseness_0  topkcloseness_1
5                1.0              1.0
10               1.0              1.0
50               1.0              1.0
100              1.0              1.0


## Spearman rank correlation

In [22]:
spearmanr_corr = {}

for k in ks:
    spearmanr_corr[k] = {}
    
    y_true = pd.DataFrame({"nodes": nodes["degree"][:k], "ranking_x": normalised_scores["degree"][:k]})

    for centrality_measure in ["closeness", "topkcloseness_0", "topkcloseness_1"]:
        
        if "topk" in centrality_measure:
            y_score = pd.DataFrame({"nodes": nodes[centrality_measure][k], "ranking_y": normalised_scores[centrality_measure][k]})
            
        else:
            y_score = pd.DataFrame({"nodes": nodes[centrality_measure][:k], "ranking_y": normalised_scores[centrality_measure][:k]})

        y_df = y_true.merge(y_score, how="outer", on="nodes").fillna(0)

        corr, _ = spearmanr(y_df["ranking_x"], y_df["ranking_y"])
        spearmanr_corr[k][centrality_measure] = corr
    
results["spearmanr"] = spearmanr_corr

print(pd.DataFrame(spearmanr_corr).T)

     closeness  topkcloseness_0  topkcloseness_1
5    -0.272727        -0.130893        -0.130893
10    0.078829         0.141239         0.141239
50   -0.009443         0.558657         0.558657
100  -0.389444        -0.310785        -0.310785
570  -0.207117        -0.196459        -0.196459


## Kendalltau rank correlation

In [23]:
kendalltau_corr = {}

for k in ks:
    kendalltau_corr[k] = {}
    
    y_true = pd.DataFrame({"nodes": nodes["degree"][:k], "ranking_x": normalised_scores["degree"][:k]})

    for centrality_measure in ["closeness", "topkcloseness_0", "topkcloseness_1"]:
        
        if "topk" in centrality_measure:
            y_score = pd.DataFrame({"nodes": nodes[centrality_measure][k], "ranking_y": normalised_scores[centrality_measure][k]})
            
        else:
            y_score = pd.DataFrame({"nodes": nodes[centrality_measure][:k], "ranking_y": normalised_scores[centrality_measure][:k]})

        y_df = y_true.merge(y_score, how="outer", on="nodes").fillna(0)

        corr, _ = kendalltau(y_df["ranking_x"], y_df["ranking_y"])
        kendalltau_corr[k][centrality_measure] = 0 if np.isnan(corr) else corr
    
results["kendalltau"] = kendalltau_corr

print(pd.DataFrame(kendalltau_corr).T)

     closeness  topkcloseness_0  topkcloseness_1
5    -0.150000        -0.052705        -0.052705
10    0.142857         0.195180         0.195180
50   -0.049360         0.489716         0.489716
100  -0.364760        -0.299510        -0.299510
570  -0.178268        -0.169867        -0.169867


# Store results

In [24]:
with open(f"../results/{results['dataset']}_{results['directed']}.json", 'w') as outfile:
    json.dump(results, outfile)